In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import QuantileTransformer
import tensorflow as tf
from pathlib import Path
from time import strftime

In [2]:
%load_ext tensorboard
%tensorboard --logdir=./my_logs
print('Tensorboard launched at localhost:6006')

Launching TensorBoard...

Tensorboard launched at localhost:6006


In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
def get_run_dir(root='my_logs'):
    return Path(root) / strftime('run_%Y_%m_%d_%H_%M_%S')

In [5]:
def get_score(true_values, predicted_values):
    mae_ = mean_absolute_error(true_values, predicted_values)
    mse_ = mean_squared_error(true_values, predicted_values)
    r2_ = r2_score(true_values, predicted_values)

    return mae_, mse_, r2_

# Loading data

In [6]:
dataset = pd.read_csv('dataset.csv')

In [7]:
dataset = dataset.loc[dataset['COND'] != '0']

data = dataset.iloc[:, :8]
labels = dataset.iloc[:, -2:]

In [8]:
data_train_full, data_test, labels_train_full, labels_test = train_test_split(data, labels, test_size=0.2, random_state=123)

data_train, data_validation, labels_train, labels_validation = train_test_split(data_train_full, labels_train_full, test_size=0.2, random_state=123)

data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 1]
data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 5]
data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 9]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 1]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 5]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 9]

data_train = data_train.loc[labels_train['ANS_VALENCE'] != 1]
data_train = data_train.loc[labels_train['ANS_VALENCE'] != 5]
data_train = data_train.loc[labels_train['ANS_VALENCE'] != 9]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 1]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 5]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 9]

# Scaling input values to Gaussian distributions

In [9]:
quantile_transformer = QuantileTransformer(output_distribution='normal', random_state=0)
data_train_qt = quantile_transformer.fit_transform(data_train)
data_train_qt = pd.DataFrame(data_train_qt, columns=data_train.columns)
data_validation_qt = quantile_transformer.fit_transform(data_validation)
data_validation_qt = pd.DataFrame(data_validation_qt, columns=data_validation.columns)

# Training

In [10]:
tf.random.set_seed(123)

norm_layer = tf.keras.layers.Normalization()

input_ = tf.keras.layers.Input(shape=data_train_qt.shape[1:])
normalized = norm_layer(input_)
hidden1 = tf.keras.layers.Dense(64, activation='relu')(normalized)
hidden2 = tf.keras.layers.Dense(128, activation='relu')(hidden1)
hidden3 = tf.keras.layers.Dense(64, activation='relu')(hidden2)
output = tf.keras.layers.Dense(2)(hidden3)

model = tf.keras.Model(inputs=[input_], outputs=[output])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 8)]               0         
                                                                 


2022-11-20 18:26:07.416985: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-20 18:26:08.344134: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3368 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1050, pci bus id: 0000:01:00.0, compute capability: 6.1


 normalization (Normalizatio  (None, 8)                17        
 n)                                                              
                                                                 
 dense (Dense)               (None, 64)                576       
                                                                 
 dense_1 (Dense)             (None, 128)               8320      
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 2)                 130       
                                                                 
Total params: 17,299
Trainable params: 17,282
Non-trainable params: 17
_________________________________________________________________


In [11]:
model.compile(loss='mse',
              optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
              metrics=['mae'])

In [12]:
norm_layer.adapt(data_train_qt)

In [13]:
n_epochs = 100

run_dir = get_run_dir()
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_dir, profile_batch=(0, n_epochs))

history = model.fit(data_train_qt, labels_train, epochs=n_epochs, validation_data=(data_validation, labels_validation), callbacks=[tensorboard_cb])

Epoch 1/100
570/570 [==============================] - 5s 6ms/step - loss: 6.9048 - mae: 2.1143 - val_loss: 7.6595 - val_mae: 2.3628
Epoch 2/100
570/570 [==============================] - 4s 7ms/step - loss: 4.2243 - mae: 1.7254 - val_loss: 7.2135 - val_mae: 2.2940
Epoch 3/100
570/570 [==============================] - 4s 7ms/step - loss: 4.1761 - mae: 1.7139 - val_loss: 6.7926 - val_mae: 2.2230
Epoch 4/100
570/570 [==============================] - 3s 5ms/step - loss: 4.1636 - mae: 1.7101 - val_loss: 6.6666 - val_mae: 2.2030
Epoch 5/100
570/570 [==============================] - 3s 5ms/step - loss: 4.1576 - mae: 1.7090 - val_loss: 6.6843 - val_mae: 2.2068
Epoch 6/100
570/570 [==============================] - 4s 7ms/step - loss: 4.1548 - mae: 1.7080 - val_loss: 6.7028 - val_mae: 2.2023
Epoch 7/100
570/570 [==============================] - 3s 5ms/step - loss: 4.1522 - mae: 1.7080 - val_loss: 6.4286 - val_mae: 2.1501
Epoch 8/100
570/570 [==============================] - 4s 7ms/step - 

In [14]:
predictions = model.predict(data_validation_qt)

mae, mse, r2 = get_score(labels_validation, predictions)
print(f'''\nValues for validation set:\nMAE: {mae}\nMSE: {mse}\nRMSE: {mse**.5}\nR2:  {r2}''')

215/215 [==============================] - 1s 2ms/step

Values for validation set:
MAE: 1.9959957686295784
MSE: 5.718140916685246
RMSE: 2.391263456143059
R2:  -0.0025440649049588426


# Overfitting?

In [15]:
tf.random.set_seed(123)

norm_layer = tf.keras.layers.Normalization()

input_ = tf.keras.layers.Input(shape=data_train_qt.shape[1:])
normalized = norm_layer(input_)
output = tf.keras.layers.Dense(2)(normalized)

model = tf.keras.Model(inputs=[input_], outputs=[output])

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 8)]               0         
                                                                 
 normalization_1 (Normalizat  (None, 8)                17        
 ion)                                                            
                                                                 
 dense_4 (Dense)             (None, 2)                 18        
                                                                 
Total params: 35
Trainable params: 18
Non-trainable params: 17
_________________________________________________________________


In [16]:
model.compile(loss='mse',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['mae'])

In [17]:
norm_layer.adapt(data_train_qt)

In [18]:
n_epochs = 50

# run_dir = get_run_dir()
# tensorboard_cb = tf.keras.callbacks.TensorBoard(run_dir, profile_batch=(0, n_epochs))

model.fit(data_train_qt, labels_train, epochs=n_epochs, validation_data=(data_validation, labels_validation))

Epoch 1/50
570/570 [==============================] - 3s 4ms/step - loss: 30.0698 - mae: 5.0194 - val_loss: 31.9420 - val_mae: 5.0728
Epoch 2/50
570/570 [==============================] - 2s 4ms/step - loss: 24.5946 - mae: 4.4764 - val_loss: 26.9238 - val_mae: 4.5637
Epoch 3/50
570/570 [==============================] - 2s 4ms/step - loss: 20.1367 - mae: 3.9702 - val_loss: 23.3622 - val_mae: 4.1753
Epoch 4/50
570/570 [==============================] - 2s 4ms/step - loss: 16.4379 - mae: 3.5189 - val_loss: 18.6946 - val_mae: 3.6982
Epoch 5/50
570/570 [==============================] - 2s 4ms/step - loss: 13.3759 - mae: 3.1330 - val_loss: 15.1614 - val_mae: 3.3217
Epoch 6/50
570/570 [==============================] - 2s 4ms/step - loss: 10.8725 - mae: 2.8067 - val_loss: 12.6602 - val_mae: 3.0349
Epoch 7/50
570/570 [==============================] - 3s 5ms/step - loss: 8.8680 - mae: 2.5349 - val_loss: 10.4132 - val_mae: 2.7704
Epoch 8/50
570/570 [==============================] - 3s 6ms/st

In [19]:
predictions = model.predict(data_validation_qt)

mae, mse, r2 = get_score(labels_validation, predictions)
print(f'''\nValues for validation set:\nMAE: {mae}\nMSE: {mse}\nRMSE: {mse**.5}\nR2:  {r2}''')

215/215 [==============================] - 1s 3ms/step

Values for validation set:
MAE: 1.9943977370435366
MSE: 5.713294992537849
RMSE: 2.3902499853650974
R2:  -0.0015593588224424026


# Check if data from all subjects works better

In [20]:
dataset = pd.read_csv('dataset_full.csv')
dataset = dataset.loc[dataset['COND'] != '0']

data = dataset.iloc[:, :8]
labels = dataset.iloc[:, -2:]
data_train_full, data_test, labels_train_full, labels_test = train_test_split(data, labels, test_size=0.2,
                                                                              random_state=123)

data_train, data_validation, labels_train, labels_validation = train_test_split(data_train_full, labels_train_full,
                                                                                test_size=0.2, random_state=123)

data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 1]
data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 5]
data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 9]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 1]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 5]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 9]

data_train = data_train.loc[labels_train['ANS_VALENCE'] != 1]
data_train = data_train.loc[labels_train['ANS_VALENCE'] != 5]
data_train = data_train.loc[labels_train['ANS_VALENCE'] != 9]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 1]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 5]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 9]

In [21]:
quantile_transformer = QuantileTransformer(output_distribution='normal', random_state=0)
data_train_qt = quantile_transformer.fit_transform(data_train)
data_train_qt = pd.DataFrame(data_train_qt, columns=data_train.columns)
data_validation_qt = quantile_transformer.fit_transform(data_validation)
data_validation_qt = pd.DataFrame(data_validation_qt, columns=data_validation.columns)

In [22]:
tf.random.set_seed(123)

norm_layer = tf.keras.layers.Normalization()

input_ = tf.keras.layers.Input(shape=data_train_qt.shape[1:])
normalized = norm_layer(input_)
output = tf.keras.layers.Dense(2)(normalized)

model = tf.keras.Model(inputs=[input_], outputs=[output])

model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 8)]               0         
                                                                 
 normalization_2 (Normalizat  (None, 8)                17        
 ion)                                                            
                                                                 
 dense_5 (Dense)             (None, 2)                 18        
                                                                 
Total params: 35
Trainable params: 18
Non-trainable params: 17
_________________________________________________________________


In [23]:
model.compile(loss='mse',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['mae'])

In [24]:
norm_layer.adapt(data_train_qt)

In [25]:
n_epochs = 50

model.fit(data_train_qt, labels_train, epochs=n_epochs, validation_data=(data_validation, labels_validation))

Epoch 1/50
2394/2394 [==============================] - 11s 5ms/step - loss: 22.3662 - mae: 4.1975 - val_loss: 17.6610 - val_mae: 3.5948
Epoch 2/50
2394/2394 [==============================] - 11s 5ms/step - loss: 9.5268 - mae: 2.6151 - val_loss: 8.6398 - val_mae: 2.5366
Epoch 3/50
2394/2394 [==============================] - 10s 4ms/step - loss: 4.9175 - mae: 1.9100 - val_loss: 6.2073 - val_mae: 2.1030
Epoch 4/50
2394/2394 [==============================] - 9s 4ms/step - loss: 4.1670 - mae: 1.7220 - val_loss: 5.8833 - val_mae: 2.0292
Epoch 5/50
2394/2394 [==============================] - 9s 4ms/step - loss: 4.1467 - mae: 1.7076 - val_loss: 5.8972 - val_mae: 2.0339
Epoch 6/50
2394/2394 [==============================] - 7s 3ms/step - loss: 4.1469 - mae: 1.7079 - val_loss: 5.8545 - val_mae: 2.0243
Epoch 7/50
2394/2394 [==============================] - 10s 4ms/step - loss: 4.1466 - mae: 1.7077 - val_loss: 5.8543 - val_mae: 2.0246
Epoch 8/50
2394/2394 [==============================] - 

In [26]:
predictions = model.predict(data_validation_qt)

mae, mse, r2 = get_score(labels_validation, predictions)
print(f'''\nValues for validation set:\nMAE: {mae}\nMSE: {mse}\nRMSE: {mse**.5}\nR2:  {r2}''')

895/895 [==============================] - 4s 4ms/step

Values for validation set:
MAE: 2.010639204864571
MSE: 5.795552315376139
RMSE: 2.4073953384054185
R2:  -0.0004854048916076992


Trying big model again

In [27]:
tf.random.set_seed(123)

norm_layer = tf.keras.layers.Normalization()

input_ = tf.keras.layers.Input(shape=data_train_qt.shape[1:])
normalized = norm_layer(input_)
hidden1 = tf.keras.layers.Dense(64, activation='relu')(normalized)
hidden2 = tf.keras.layers.Dense(128, activation='relu')(hidden1)
hidden3 = tf.keras.layers.Dense(64, activation='relu')(hidden2)
output = tf.keras.layers.Dense(2)(hidden3)

model = tf.keras.Model(inputs=[input_], outputs=[output])

model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 8)]               0         
                                                                 
 normalization_3 (Normalizat  (None, 8)                17        
 ion)                                                            
                                                                 
 dense_6 (Dense)             (None, 64)                576       
                                                                 
 dense_7 (Dense)             (None, 128)               8320      
                                                                 
 dense_8 (Dense)             (None, 64)                8256      
                                                                 
 dense_9 (Dense)             (None, 2)                 130       
                                                           

In [28]:
model.compile(loss='mse',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['mae'])
norm_layer.adapt(data_train_qt)

In [29]:
n_epochs = 25

model.fit(data_train_qt, labels_train, epochs=n_epochs, validation_data=(data_validation, labels_validation))

Epoch 1/25
2394/2394 [==============================] - 23s 9ms/step - loss: 4.4340 - mae: 1.7525 - val_loss: 6.8912 - val_mae: 2.2445
Epoch 2/25
2394/2394 [==============================] - 22s 9ms/step - loss: 4.1782 - mae: 1.7140 - val_loss: 6.5386 - val_mae: 2.1814
Epoch 3/25
2394/2394 [==============================] - 22s 9ms/step - loss: 4.1673 - mae: 1.7116 - val_loss: 6.8210 - val_mae: 2.2431
Epoch 4/25
2394/2394 [==============================] - 21s 9ms/step - loss: 4.1631 - mae: 1.7106 - val_loss: 6.5667 - val_mae: 2.1858
Epoch 5/25
2394/2394 [==============================] - 20s 8ms/step - loss: 4.1582 - mae: 1.7099 - val_loss: 7.3545 - val_mae: 2.3278
Epoch 6/25
2394/2394 [==============================] - 20s 8ms/step - loss: 4.1558 - mae: 1.7091 - val_loss: 6.6870 - val_mae: 2.2156
Epoch 7/25
2394/2394 [==============================] - 20s 8ms/step - loss: 4.1500 - mae: 1.7081 - val_loss: 6.7798 - val_mae: 2.2267
Epoch 8/25
2394/2394 [==============================] -

In [30]:
predictions = model.predict(data_validation_qt)

mae, mse, r2 = get_score(labels_validation, predictions)
print(f'''\nValues for validation set:\nMAE: {mae}\nMSE: {mse}\nRMSE: {mse**.5}\nR2:  {r2}''')

895/895 [==============================] - 2s 2ms/step

Values for validation set:
MAE: 2.0060391341058175
MSE: 5.83354493798229
RMSE: 2.4152732636251097
R2:  -0.0033273456713504568
